In [1]:
## Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.utils import class_weight
from collections import Counter
import torch
import numpy as np
import evaluate

## Load the dataset

In [2]:
# === Load CSV ===
dataset = load_dataset("csv", data_files="./AI_Human.csv")

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized = dataset.map(tokenize_fn, batched=True)
tokenized = tokenized.rename_column("generated", "labels")
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [3]:
# === Compute class weights ===
labels = dataset["train"]["generated"]
classes = np.unique(labels)
weights = class_weight.compute_class_weight("balanced", classes=classes, y=labels)
class_weights = torch.tensor(weights, dtype=torch.float)

print("Class distribution:", Counter(labels))
print("Class weights:", class_weights)

Class distribution: Counter({0.0: 305797, 1.0: 181438})
Class weights: tensor([0.7967, 1.3427])


In [4]:
# === Load RoBERTa-large with LoRA ===
base_model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

model = get_peft_model(base_model, lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# === Custom loss with class weights ===
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fn(logits, labels)
    return (loss, outputs) if return_outputs else loss

# === Evaluation metrics ===
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }


In [6]:
# === Training arguments ===
training_args = TrainingArguments(
    output_dir="./roberta-large-peft-ai-human",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    load_best_model_at_end=True,
    fp16=True,
)

In [7]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").long()  
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [8]:
# === Split train/validation ===
split = tokenized["train"].train_test_split(test_size=0.1, seed=42)
train_ds = split["train"]
eval_ds = split["test"]


trainer = CustomTrainer(  # use custom trainer defined below
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,  # FutureWarning: will be removed — still works for now
    compute_metrics=compute_metrics
)

C:\Users\Dayso\AppData\Local\Temp\ipykernel_29924\3427178716.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(  # use custom trainer defined below
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
# === Train ===
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.041600,0.063777,0.987296,0.983318
2,0.000100,0.091543,0.984771,0.980099
3,0.000000,0.058418,0.991236,0.988454


c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2b328107-278c-42f0-aa4c-d710488550e1)') - silently ignoring the lookup for the file config.json in roberta-large.
  warnings.warn(
c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\save_and_load.py:238: UserWarning: Could not find a config file in roberta-large - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2fb9407b-5ed1-4ded-bfcd-f843dd2cde9d)') - silently ignoring the lookup for the file config.json

TrainOutput(global_step=164442, training_loss=0.012417587256003093, metrics={'train_runtime': 27289.305, 'train_samples_per_second': 48.207, 'train_steps_per_second': 6.026, 'total_flos': 6.167073674590433e+17, 'train_loss': 0.012417587256003093, 'epoch': 3.0})

In [ ]:
# save the model

In [10]:
# === Save PEFT adapter + tokenizer ===
ADAPTER_DIR = "./saved_large/roberta_peft_adapter"
TOKENIZER_DIR = ADAPTER_DIR  # save tokenizer alongside for convenience

# make sure trainer.model is your trained PEFT model
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(TOKENIZER_DIR)

print(f"Saved PEFT adapter + tokenizer to: {ADAPTER_DIR}")

Saved PEFT adapter + tokenizer to: ./saved_large/roberta_peft_adapter


In [11]:
# === Merge LoRA into base weights and save a standard model ===
from copy import deepcopy

MERGED_DIR = "./saved_large/roberta_merged_full"

# work on a copy so the trainer's in-memory model keeps its PEFT structure
merged = deepcopy(trainer.model)
merged = merged.merge_and_unload()  # folds LoRA weights into the base model

merged.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print(f"Saved merged full model + tokenizer to: {MERGED_DIR}")


Saved merged full model + tokenizer to: ./saved_large/roberta_merged_full


In [ ]:
from sklearn.metrics import classification_report
# Final eval
trainer.evaluate()

# Classification report
preds = trainer.predict(tokenized["test"])
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

print(classification_report(y_true, y_pred, target_names=["Human", "AI"]))
